In [ ]:
!python -m pip install seaborn

In [ ]:
!python -m pip install scikit-learn

In [6]:
from elasticsearch import Elasticsearch
client = Elasticsearch("http://localhost:9200")
resp = client.search(index="retromotion-indexer_development_products",
                     body={"_source":["descriptions","descriptionsSource","nameSource","shortDescriptionSource","categoriesSource"],
                           "query": {"match_all": {}}})


In [4]:
print("Got %d Hits:" % resp['hits']['total'])

Got 22329 Hits:


In [5]:
import pandas as pd
import re
import numpy as np

import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
df_eng = pd.DataFrame(columns=['name','sdesc','category','catlevel0','catlevel1','catlevel2','catlevel3','catlevel4','catlevel5'])
df_de = pd.DataFrame(columns=['name','sdesc','desc','category','categorypath'])

In [11]:
df_eng.head()

,name,sdesc,category,catlevel0,catlevel1,catlevel2,catlevel3,catlevel4,catlevel5


In [12]:
def cleanStr(text):
   
    # remove numbers
    # no_number_string = re.sub(r'\d+','',lower_string)

    # remove all punctuation except words and space
    # text = re.sub(r'[^\w\s]','', lower_string)

    # clean = re.compile('<.*?>')

    # text = re.sub(clean,'',text)
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' .join(emoticons).replace('-', ''))

    return text


In [13]:
def readHtml(html):
    dftable = pd.read_html(html)
    return dftable

In [14]:
name_en=[]
name_de=[]

sdesc_en=[]
sdesc_de=[]

desc_en=[]
desc_de=[]

cat_en=[]
cat_de=[]

catpath_en=[]
catpath_de=[]

for hit in resp['hits']['hits']:
    list_row =[]
    for name in hit['_source']['nameSource']:
        if (name["language"]=="en"):
            list_row.append(cleanStr(name["value"]))
        else:
            name_de.append(name["value"])
    
    for sdesc in hit['_source']['shortDescriptionSource']:
        if (sdesc["language"]=="en"):
            list_row.append(cleanStr(sdesc["value"]))
        else:
            sdesc_de.append(sdesc["value"])

    # for desc in hit['_source']['descriptionsSource']:
    #     if (desc["language"]=="en"):
    #         desc_en.append(readHtml(desc["value"]))
    #     else:
    #         desc_de.append(desc["value"])
    
    for cats in hit['_source']['categoriesSource']:
        if (cats["language"]=="en"):
            list_row.append(cleanStr(cats["label"]))
        else:
            desc_de.append(cats["label"])

        if (cats["language"]=="en"):
            categories =cats["path"].split('/')
            i =0
            catls=[]
            for category in categories:

                list_row.append(cleanStr(category))
                i= i+1
            
        else:
            catpath_de.append(cats["path"])
    
    # new_row = pd.DataFrame({'name':_name, 'sdesc':_sdesc, 'category':_cat_en }, index=[0])

    # pd.concat([new_row,df_eng.loc[:]]).reset_index(drop=True)

    for i in range (0,9-len(list_row)):
        list_row.append(np.nan)

    df_eng.loc[len(df_eng)] = list_row



In [ ]:
# df_en = pd.DataFrame({'name':name_en,
#                        'sdesc':sdesc_en,
#                     #    'desc':desc_en,
#                        'category':cat_en,
#                        'categorypath':catpath_en})


# df_de = pd.DataFrame({'name':name_de,
#                        'sdesc':sdesc_de,
#                        'desc':desc_de,
#                        'category':cat_de,
#                        'categorypath':catpath_de})

In [15]:
df_eng.head()

,name,sdesc,category,catlevel0,catlevel1,catlevel2,catlevel3,catlevel4,catlevel5
0,denso dtm82363 thermostat coolant,thermostat coolant,thermostat,sparepart,cooling system,thermostat,NaN,NaN,NaN
1,zf 1043 010 297 automatic transmission,automatic transmission,automatic transmission,sparepart,transmission,automatic transmission,automatic transmission,NaN,NaN
2,zf 1060 040 003 automatic transmission,automatic transmission,automatic transmission,sparepart,transmission,automatic transmission,automatic transmission,NaN,NaN
3,vaico v10 0131 camshaft,camshaft,camshaft,sparepart,engine,engine timing,camshaft,NaN,NaN
4,vaico v26 0302 track control arm,track control arm,track control arm,sparepart,powertrain suspension,wheel suspension,track control arm,NaN,NaN


In [ ]:
df_eng.isna().sum()

In [ ]:
df_eng.shape

In [ ]:
df_eng = df_eng.dropna()

In [ ]:
df_eng.shape

In [ ]:
df_eng.head()

In [ ]:
df_eng.describe()

One hot encoding 

In [ ]:
df_eng = pd.get_dummies(df_eng,columns=["category"])
df_eng.head()

In [ ]:
df_eng.shape

In [ ]:
from sklearn.model_selection import train_test_split

